# add addl optional references to subject in SD

1. get list of SD that have subject element
2. read SD from resources-yaml folder

   1. add ms extension to Patient reference of subject element
   2. find other reference types from base
   3. add to Subject element in profile
   4. save as yaml (sort keys)

In [49]:
from yaml import load as yload, dump as ydump, Loader
from json import loads, dumps
from pathlib import Path
from copy import deepcopy
from datetime import datetime
from fhir.resources.structuredefinition import StructureDefinition
from pprint import pprint

# my_path = r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input'
my_path = r'/Users/ehaas/Documents/FHIR/US-Core/input'
in_path = Path(my_path) / 'resources-yaml'
out_path = Path(my_path) / 'resources-yaml-copy'
out_path.mkdir(parents=True, exist_ok=True)
#create out_path using pathlib Path.mkdir(parents=True, exist_ok=True)

R4_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package')

Types = ['CarePlan',
'CareTeam',
'Condition',
'DiagnosticReport',
'DocumentReference',
'Encounter',
'Goal',
'MedicationDispense',
'MedicationRequest',
'Observation',
'Observation',
'Procedure',
'QuestionnaireResponse',
'ServiceRequest',
'Specimen',
]

_targetProfile = [
        {
        "extension": [
            {
                "url": "http://hl7.org/fhir/StructureDefinition/elementdefinition-type-must-support",
                "valueBoolean": True
            }
            ]
        },
    ]

today = datetime.today().strftime('%Y-%m-%d')
today, in_path, out_path

('2023-11-03',
 PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml'),
 PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml-copy'))

In [50]:
def get_targetProfile(r_type, element_path):
    for sd_base in R4_path.glob(f'StructureDefinition*-{r_type}.json'):
        try:
            r_base = loads(sd_base.read_text())
        except Exception as e:
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
        # print(f"======{sd_base}, {r_base['type']}======")
        for element in r_base['differential']['element']:
            if element['path'] == element_path:
                return element["type"][0]["targetProfile"]
                              
        # print(f'❗❗❗element not found in base SD - look at the datatype in the path {element_path}')
        # #use the penultimate element in element_path to find the datatype in the base SD
        # parent_element_path = element_path.rsplit('.', 1)[0]
        # #use the last two element in element_path to find the datatype element in the base SD
        # datatype_element_path = element_path.split('.', )[-1]

        # print(f"parent_element_path = {parent_element_path} datatype_element_path = {datatype_element_path}")

        # #get the datatype
        # for element in r_base['differential']['element']:
        #     if element['id'] == parent_element_path:
        #         for type in element['type']:
        #             print(f"datatype = {type['code']}")
        #             #get the datatype SD short and compare to the shorty
        #             path = f"{type['code']}.{datatype_element_path}"
        #             print(f"type = {type['code']}, path = {path}")
        #             return get_shorty(type['code'], (f"{type['code']}.{datatype_element_path}"),)

target_list = []
# read StructureDefinitions from resources-yaml folder using pathlib
for sd in in_path.glob('Struct*.yml'):
    print(f"***********{sd}***********")
    r = yload(sd.read_text(), Loader=Loader)
    r_copy = deepcopy(r)
    print(f"☞ ☞ ☞ Type = {r['type']}")
    if r['type'] not in Types:
        print(f"❗❗❗Type {r['type']} not in Types")
        continue
    try:
        subject = (element for element in r['differential']['element']
                        if element['path'].split('.')[-1] == 'subject').__next__()
        print(f'☞ ☞ ☞ subject ref types = {subject["type"][0]["targetProfile"]}')
        R4_targetProfile = get_targetProfile(r_type=r['type'], element_path=subject['path'])
        print(f'☞ ☞ ☞ subject R4 types = {R4_targetProfile}')
        for target_Profile in R4_targetProfile:
            if target_Profile == 'http://hl7.org/fhir/StructureDefinition/Patient':
                print(f"❗❗❗target_Profile {target_Profile} is Patient")
                continue
            if target_Profile == 'http://hl7.org/fhir/StructureDefinition/Location':
                subject["type"][0]["targetProfile"].append('http://hl7.org/fhir/us/core/StructureDefinition/us-core-location')
                continue
            subject["type"][0]["targetProfile"].append(target_Profile)
        subject["type"][0]["_targetProfile"] = _targetProfile  
        target_list = target_list  + (R4_targetProfile)

    except StopIteration:
        print(f"❗❗❗No subject element")
        continue
    pprint(f"☞ ☞ ☞ subject = {subject}")
    # custom sort the keys in the elements using fhir.resources
    r = StructureDefinition.parse_obj(r)
    # print(r.json(indent=4))
        # save the copy to out_path using Pathlib
    copyfile = out_path / sd.name
    copyfile.write_text(r.yaml(indent=2))

***********/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-sex.yml***********
☞ ☞ ☞ Type = Extension
❗❗❗Type Extension not in Types
***********/Users/ehaas/Documents/FHIR/US-Core/input/resources-yaml/StructureDefinition-us-core-specimen.yml***********
☞ ☞ ☞ Type = Specimen
☞ ☞ ☞ subject ref types = ['http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient']
☞ ☞ ☞ subject R4 types = ['http://hl7.org/fhir/StructureDefinition/Patient', 'http://hl7.org/fhir/StructureDefinition/Group', 'http://hl7.org/fhir/StructureDefinition/Device', 'http://hl7.org/fhir/StructureDefinition/Substance', 'http://hl7.org/fhir/StructureDefinition/Location']
❗❗❗target_Profile http://hl7.org/fhir/StructureDefinition/Patient is Patient
("☞ ☞ ☞ subject = {'id': 'Specimen.subject', 'path': 'Specimen.subject', "
 "'short': 'The patient where the specimen came from.', 'type': [{'code': "
 "'Reference', 'targetProfile': "
 "['http://hl7.org/fhir/us/core/StructureDefinition/

In [51]:
%%bash -s "$in_path" "$out_path"
##### DO THIS ONLY when you want to overwrite resources-yaml folder with resources-yaml-copy ####

cp $2/Struct*.yml $1